In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder \
     .appName("SparkSession") \
     .getOrCreate()

sc = spark.sparkContext 
sc

<SparkContext master=yarn appName=SparkSession>

In [2]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [4]:
data = spark.read.json("s3://qianyielva/product")
data.show(10)

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|0000000078|  [1, 1]|    5.0|Conversations wit...|08 11, 2004|A3AF8FFZAZYNE5|                null|          Impactful!|    1092182400|
|0000000116|  [5, 5]|    4.0|Interesting Grish...|04 27, 2002| AH2L9G3DQHHAJ|               chris|  Show me the money!|    1019865600|
|0000000116|  [0, 0]|    1.0|The thumbnail is ...|03 24, 2014|A2IIIDRK3PRRZY|              Helene|Listing is all sc...|    1395619200|
|0000000868|[10, 10]|    4.0|I'll be honest. I...|09 11, 2002|A1TADCM7YWPQ8M|            Joel@AWS|Not a Bad Transla...|    1031702400|
|0000013714|  [0, 0]|    4.0|It had all the so...|10 31

In [5]:
drop_list = ['asin', 'helpful', 'reviewTime', 'reviewerID', 'reviewerName', 'summary', 'unixReviewTime']
data = data.select([column for column in data.columns if column not in drop_list])
data.createOrReplaceTempView("data")
data=spark.sql("SELECT overall, LOWER(reviewText) AS reviewText FROM data")
data.show(5)
data.printSchema()

+-------+--------------------+
|overall|          reviewText|
+-------+--------------------+
|    5.0|conversations wit...|
|    4.0|interesting grish...|
|    1.0|the thumbnail is ...|
|    4.0|i'll be honest. i...|
|    4.0|it had all the so...|
+-------+--------------------+
only showing top 5 rows

root
 |-- overall: double (nullable = true)
 |-- reviewText: string (nullable = true)



In [ ]:
from pyspark.sql.functions import col
data.groupBy("overall") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

data.groupBy("reviewText") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+-------+--------+
|overall|   count|
+-------+--------+
|    5.0|49169670|
|    4.0|15480820|
|    3.0| 7049302|
|    1.0| 6712117|
|    2.0| 4265230|
+-------+--------+



In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt')
import nltk
from nltk.corpus import stopwords
stopwords_lst=stopwords.words('english')+['1','2','3','4','5','6','7','8','9','0',]

In [ ]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="reviewText", outputCol="words", pattern="\\W")
# stop words
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(stopwords_lst)
# bag of words count
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=1)

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
label_stringIdx = StringIndexer(inputCol = "overall", outputCol = "label")
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])
# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(data)
dataset = pipelineFit.transform(data)
dataset.show(5)

label_stringIdx: over->label 1=3, 2=4, 3=2, 4=1, 5=0

In [ ]:
# set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

NaiveBayes

In [ ]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(smoothing=1)
model = nb.fit(trainingData)
predictions = model.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("reviewText","overall","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

Radom Forest

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32)
# Train model with Training Data
rfModel = rf.fit(trainingData)
predictions = rfModel.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("reviewText","overall","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

In [ ]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

3 label data

In [ ]:
data.createOrReplaceTempView('data')

In [ ]:
# overall=1 or 2, attitue =1 (negative) 
# overall=3, attitue =2 (neural)
# overall=4 or 5, attitue =3 (positive)
from pyspark.sql import functions as F
df=data.withColumn('attitude', F.when(F.col('overall')<3,'negative').otherwise(F.when( F.col('overall') == 3,'neutral').otherwise('positive')))

In [ ]:
df.show(10)
df.groupBy("attitude") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

In [ ]:
label_stringIdx = StringIndexer(inputCol = "attitude", outputCol = "label")
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])
# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(df)
dataset = pipelineFit.transform(df)
dataset.show(5)

In [ ]:
# set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

In [ ]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(smoothing=1)
model = nb.fit(trainingData)
predictions = model.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("reviewText","attitude","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

Random Forest

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32)
# Train model with Training Data
rfModel = rf.fit(trainingData)
predictions = rfModel.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("reviewText","attitude","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

In [ ]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)